In [1]:
import polars as pl
import os

In [2]:
#lets read in the data I downloaded from hugging face, I start with the last 6 files
data_folder = "data/"
df1 = pl.read_parquet(os.path.join(data_folder, "train-00121-of-00127.parquet"))
df2 = pl.read_parquet(os.path.join(data_folder, "train-00122-of-00127.parquet"))
df3 = pl.read_parquet(os.path.join(data_folder, "train-00123-of-00127.parquet"))
df4 = pl.read_parquet(os.path.join(data_folder, "train-00124-of-00127.parquet"))
df5 = pl.read_parquet(os.path.join(data_folder, "train-00125-of-00127.parquet"))
df6 = pl.read_parquet(os.path.join(data_folder, "train-00126-of-00127.parquet"))

In [3]:
# let's take a look at the first few columns and the column names
df1.head()

repo,issue_id,issue_number,pull_request,events,text_size,content,usernames
str,i64,i64,struct[3],list[struct[8]],i64,str,str
"""kaisermann/svelte-i18n""",550510104,40,"{40,""svelte-i18n"",""kaisermann""}","[{""opened"",""kaisermann"",null,""2020-01-16T00:48:55Z"",""username_0"","""",""WIP - v3"",""issue""}, {""created"",""elbourki"",580035734,""2020-01-30 01:00:41+00:00"",""username_1"",""Hey! That's a pretty awesome package you got there! What if we removed `getClientLocale` from utils and made it 100% optional? We could do that by making `initialLocale` accept an array of _ISO_ codes and it finds the first one that has registered translations... I think this way it would allow developers to use external libraries like _wojtekmaj/get-user-locale_, they offer a pretty handy method called `getUserLocales` that returns an array as well, so this means you won't have to maintain that functionality and developers would have more flexibility. And please let me know if you have a to do list or something, I can help with some tasks :)"",null,""comment""}, … {""created"",""elbourki"",586596649,""2020-02-15 14:40:59+00:00"",""username_1"",""Hey Chris! I apologize for not being able to help with 3.0, I got a bit busier in the last few weeks :( I was thinking of replacing `intl-messageformat` with `message-format`, it's at 21.5kB compared to 29.8kB, not a huge improvement but it would reduce the library size to 28.4kB 😬 But note that they don't have the same [API](https://github.com/format-message/format-message/tree/master/packages/message-format#messageformat), we'd have to remove the _formats_ option if we ever choose to adopt this library. Another interesting option would be [Frenchkiss.js](https://github.com/koala-interactive/frenchkiss.js), it supports ICU and it's currently at 2.7kB 😱. The way the library works is quite different and we will probably need a major rewrite to implement it, but I think it's worth it :D"",null,""comment""}]",2398,"""<issue_start><issue_comment>Ti…","""[""kaisermann"", ""elbourki""]"""
"""TheJacksonLaboratory/GLASS""",550594847,153,null,"[{""opened"",""Kevin-Moo"",null,""2020-01-16 06:07:35+00:00"",""username_0"",""Dear Floris, Thank you very much for making the code and data of the GLASS project open! It is a really exciting and important endeavor. I am trying to reproduce Figure 3A from the open data, and I encountered some problems when selecting the patients. I wonder if it is possible for you to provide the ordered list of patients in Figure 3A. Thanks! Best regards, Quanhua Mu PhD student Hong Kong University of Science and Technology"",""Patient list in Figure 3A"",""issue""}, {""created"",""fpbarthel"",586480386,""2020-02-14 21:30:07+00:00"",""username_1"",""Dear Kevin, Thank you for your interest and my apologies for the late reply. The figure is roughly sorted by the number of drivers per patient. I cannot provide you with an exact list of patients, but I can refer you to the R code that was used to generate the figures here https://github.com/TheJacksonLaboratory/GLASS/blob/master/R/figures/F1-F3-GLASS-heatmap.R. This R code references SQL queries that were used to build the input data tables and these were subsequently sorted in R. I hope that this helps. Best regards, Floris"",null,""comment""}, {""closed"",""fpbarthel"",null,""2020-02-14 21:30:09+00:00"",""username_1"","""",null,""issue""}]",985,"""<issue_start><issue_comment>Ti…","""[""Kevin-Moo"", ""fpbarthel""]"""
"""jaegertracing/jaeger-operator""",550687388,854,null,"[{""opened"",""yogeek"",null,""2020-01-16 09:41:27+00:00"",""username_0"",""Hi, When installing the operator according to the documentation, the fact that `kubectl create` is used causes the command to not be idempotent : ``` $ kubectl create -f https://raw.githubusercontent.com/jaegertracing/jaeger-operator/master/deploy/crds/jaegertracing.io_jaegers_crd.yaml customresourcedefinition.apiextensions.k8s.io/jaegers.jaegertracing.io created $ kubectl create -f https://raw.gi

In [4]:
# let's filter out issues without pull requests
onlyPR1 = df1.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR2 = df2.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR3 = df3.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR4 = df4.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR5 = df5.filter(pl.col("pull_request")!="{null,null,null}")
onlyPR6 = df6.filter(pl.col("pull_request")!="{null,null,null}")

In [5]:
# let's squash it into one df
pr_df = pl.concat([onlyPR1,onlyPR2,onlyPR3,onlyPR4,onlyPR5,onlyPR6])

In [6]:
# that's a pretty big nice dataset
pr_df.shape

(478525, 8)

In [7]:
# to compare the Open Issues to the issues from our dataset, we extract what's between
# the first <issue comment> and second <issue comment> and save it in "issue" column
# since that's the title and the description of the issue and will be crucial in estimating the similarity between
# Open issues and issue from our dataset
pr_df

shape: (478_525, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ repo       ┆ issue_id   ┆ issue_numb ┆ pull_requ ┆ events    ┆ text_size ┆ content   ┆ usernames │
│ ---        ┆ ---        ┆ er         ┆ est       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆ i64        ┆ ---        ┆ ---       ┆ list[stru ┆ i64       ┆ str       ┆ str       │
│            ┆            ┆ i64        ┆ struct[3] ┆ ct[8]]    ┆           ┆           ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ kaisermann ┆ 550510104  ┆ 40         ┆ {40,"svel ┆ [{"opened ┆ 2398      ┆ <issue_st ┆ ["kaiserm │
│ /svelte-i1 ┆            ┆            ┆ te-i18n", ┆ ","kaiser ┆           ┆ art><issu ┆ ann", "el │
│ 8n         ┆            ┆            ┆ "kaiserma ┆ mann",nul ┆           ┆ e_comment ┆ bourki"]  │
│            ┆            ┆            ┆ nn"…      ┆ l,"…      ┆           ┆ >Ti…      ┆           │
│ material-c ┆ 551064006  ┆ 9444       ┆ {9444,"ma ┆ [{"opened ┆ 355       ┆ <issue_st ┆ ["jverkoe │
│ omponents/ ┆            ┆            ┆ terial-co ┆ ","jverko ┆           ┆ art><issu ┆ y", "brya │
│ material-c ┆            ┆            ┆ mponents- ┆ ey",null, ┆           ┆ e_comment ┆ noltman"] │
│ …          ┆            ┆            ┆ ios…      ┆ "20…      ┆           ┆ >Ti…      ┆           │
│ hmcts/ccd- ┆ 551466679  ┆ 758        ┆ {758,"ccd ┆ [{"opened ┆ 601       ┆ <issue_st ┆ ["noronha │
│ data-store ┆            ┆            ┆ -data-sto ┆ ","noronh ┆           ┆ art><issu ┆ a"]       │
│ -api       ┆            ┆            ┆ re-api"," ┆ aa",null, ┆           ┆ e_comment ┆           │
│            ┆            ┆            ┆ hmc…      ┆ "20…      ┆           ┆ >Ti…      ┆           │
│ dlang/phob ┆ 551980198  ┆ 7355       ┆ {7355,"ph ┆ [{"opened ┆ 4306      ┆ <issue_st ┆ ["wilzbac │
│ os         ┆            ┆            ┆ obos","dl ┆ ","jacob- ┆           ┆ art><issu ┆ h", "etci │
│            ┆            ┆            ┆ ang"}     ┆ carlborg" ┆           ┆ e_comment ┆ mon",     │
│            ┆            ┆            ┆           ┆ ,nu…      ┆           ┆ >Ti…      ┆ "Geod2…   │
│ patternfly ┆ 552466427  ┆ 686        ┆ {686,"pat ┆ [{"opened ┆ 1995      ┆ <issue_st ┆ ["starrye │
│ /patternfl ┆            ┆            ┆ ternfly-e ┆ ","Lyndse ┆           ┆ art><issu ┆ yez024",  │
│ y-elements ┆            ┆            ┆ lements", ┆ yR",null, ┆           ┆ e_comment ┆ "LyndseyR │
│            ┆            ┆            ┆ "pa…      ┆ "20…      ┆           ┆ >Ti…      ┆ "]        │
│ …          ┆ …          ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         ┆ …         │
│ okex/excha ┆ 1187862225 ┆ 1786       ┆ {1786,"ex ┆ [{"opened ┆ 1092      ┆ <issue_st ┆ ["cwbhhjl │
│ in         ┆            ┆            ┆ chain","o ┆ ","cwbhhj ┆           ┆ art><issu ┆ "]        │
│            ┆            ┆            ┆ kex"}     ┆ l",null," ┆           ┆ e_comment ┆           │
│            ┆            ┆            ┆           ┆ 202…      ┆           ┆ >Ti…      ┆           │
│ rust-bitco ┆ 1188051396 ┆ 924        ┆ {924,"rus ┆ [{"opened ┆ 2842      ┆ <issue_st ┆ ["sanket1 │
│ in/rust-bi ┆            ┆            ┆ t-bitcoin ┆ ","dr-orl ┆           ┆ art><issu ┆ 729", "dr │
│ tcoin      ┆            ┆            ┆ ","rust-b ┆ ovsky",nu ┆           ┆ e_comment ┆ -orlovsky │
│            ┆            ┆            ┆ itc…      ┆ ll,…      ┆           ┆ >Ti…      ┆ "]        │
│ funcx-faas ┆ 1188111034 ┆ 747        ┆ {747,"fun ┆ [{"opened ┆ 427       ┆ <issue_st ┆ ["sirosen │
│ /funcX     ┆            ┆            ┆ cX","func ┆ ","khk-gl ┆           ┆ art><issu ┆ ", "khk-g │
│            ┆            ┆            ┆ x-faas"}  ┆ obus",nul ┆           ┆ e_comment ┆ lobus"]   │
│            ┆            ┆            ┆           ┆ l,"…      ┆           ┆ >Ti…      ┆           │
│ MicrosoftD ┆ 1188456644 ┆ 5640       ┆ 

In [8]:
# regex on friday night, yeah!
import re

def extract_text(text): 
    pattern = r'<issue_comment>(.*?)<issue_comment>' 
    match = re.search(pattern, text, re.DOTALL) 
    if match: 
        return match.group(1) 
    else: 
        return None

In [9]:
pr_df = pr_df.with_columns([ pl.col("content").map_elements(extract_text, return_dtype=pl.Utf8).alias("issue") ])

In [10]:
pr_df.head()
#and it worked!

repo,issue_id,issue_number,pull_request,events,text_size,content,usernames,issue
str,i64,i64,struct[3],list[struct[8]],i64,str,str,str
"""kaisermann/svelte-i18n""",550510104,40,"{40,""svelte-i18n"",""kaisermann""}","[{""opened"",""kaisermann"",null,""2020-01-16T00:48:55Z"",""username_0"","""",""WIP - v3"",""issue""}, {""created"",""elbourki"",580035734,""2020-01-30 01:00:41+00:00"",""username_1"",""Hey! That's a pretty awesome package you got there! What if we removed `getClientLocale` from utils and made it 100% optional? We could do that by making `initialLocale` accept an array of _ISO_ codes and it finds the first one that has registered translations... I think this way it would allow developers to use external libraries like _wojtekmaj/get-user-locale_, they offer a pretty handy method called `getUserLocales` that returns an array as well, so this means you won't have to maintain that functionality and developers would have more flexibility. And please let me know if you have a to do list or something, I can help with some tasks :)"",null,""comment""}, … {""created"",""elbourki"",586596649,""2020-02-15 14:40:59+00:00"",""username_1"",""Hey Chris! I apologize for not being able to help with 3.0, I got a bit busier in the last few weeks :( I was thinking of replacing `intl-messageformat` with `message-format`, it's at 21.5kB compared to 29.8kB, not a huge improvement but it would reduce the library size to 28.4kB 😬 But note that they don't have the same [API](https://github.com/format-message/format-message/tree/master/packages/message-format#messageformat), we'd have to remove the _formats_ option if we ever choose to adopt this library. Another interesting option would be [Frenchkiss.js](https://github.com/koala-interactive/frenchkiss.js), it supports ICU and it's currently at 2.7kB 😱. The way the library works is quite different and we will probably need a major rewrite to implement it, but I think it's worth it :D"",null,""comment""}]",2398,"""<issue_start><issue_comment>Ti…","""[""kaisermann"", ""elbourki""]""","""Title: WIP - v3 username_0: """
"""material-components/material-c…",551064006,9444,"{9444,""material-components-ios"",""material-components""}","[{""opened"",""jverkoey"",null,""2020-01-16T21:25:40Z"",""username_0"",""[AppBar] Fix swipe to go back gesture for MDCAppBarNavigationController. https://developer.apple.com/design/human-interface-guidelines/ios/user-interaction/gestures/ ""But users can also navigate back by swiping from the side of the screen."""",""[AppBar] Fix swipe to go back gesture for MDCAppBarNavigationController."",""issue""}, {""created"",""bryanoltman"",575356739,""2020-01-16 21:32:17+00:00"",""username_1"",""Did you mean to merge this into `develop`?"",null,""comment""}, {""created"",""jverkoey"",575357719,""2020-01-16 21:34:52+00:00"",""username_0"",""Yep; will update once ready for proper review. Fixing attributes first."",null,""comment""}]",355,"""<issue_start><issue_comment>Ti…","""[""jverkoey"", ""bryanoltman""]""","""Title: [AppBar] Fix swipe to g…"
"""hmcts/ccd-data-store-api""",551466679,758,"{758,""ccd-data-store-api"",""hmcts""}","[{""opened"",""noronhaa"",null,""2020-01-17T15:06:13Z"",""username_0"",""**Before creating a pull request make sure that:** - [ ] commit messages are meaningful and follow good commit message guidelines - [ ] README and other documentation has been updated / added (if needed) - [ ] tests have been updated / new tests has been added (if needed) Please remove this line and everything above and fill the following sections: ### JIRA link (if applicable) ### https://tools.hmcts.net/jira/browse/RDM-7289 ### Change description ### Update jackson-databind **Does this PR introduce a breaking change?** (check one with ""x"") ``` [ ] Yes [x] No ```"",""RDM-7289 - Update jackson-databind"",""issue""}]",601,"""<issue_start><issue_comment>Ti…","""[""noronhaa""]""",null
"""dlang/phobos""",551980198,7355,"{7355,""phobos"",""dlang""}","[{""opened"",""jacob-carlborg""

In [11]:
# since I used <issue_comment> as the second delimiter it produced output only for Issues with at least one comment
# and there were some null values from issues with only titles and descriptions, so let's leave it like that
# and remove the null values
pr_df = pr_df.filter(pl.col("issue").is_not_null())
pr_df.shape

(414185, 9)

In [12]:
pr_df.columns

['repo',
 'issue_id',
 'issue_number',
 'pull_request',
 'events',
 'text_size',
 'content',
 'usernames',
 'issue']

In [13]:
# lets now split information in "repo" column into two separate columns, specifically "parent_repo" and "child_repo"
pr_df = pr_df.with_columns(pr_df.select(pl.col("repo").str.split(by="/").list.to_struct(n_field_strategy="max_width")).unnest("repo"))

In [14]:
pr_df

shape: (414_185, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ repo      ┆ issue_id  ┆ issue_num ┆ pull_requ ┆ … ┆ usernames ┆ issue     ┆ field_0   ┆ field_1  │
│ ---       ┆ ---       ┆ ber       ┆ est       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ i64       ┆ ---       ┆ ---       ┆   ┆ str       ┆ str       ┆ str       ┆ str      │
│           ┆           ┆ i64       ┆ struct[3] ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ kaiserman ┆ 550510104 ┆ 40        ┆ {40,"svel ┆ … ┆ ["kaiserm ┆ Title:    ┆ kaiserman ┆ svelte-i │
│ n/svelte- ┆           ┆           ┆ te-i18n", ┆   ┆ ann", "el ┆ WIP - v3  ┆ n         ┆ 18n      │
│ i18n      ┆           ┆           ┆ "kaiserma ┆   ┆ bourki"]  ┆ username_ ┆           ┆          │
│           ┆           ┆           ┆ nn"…      ┆   ┆           ┆ 0:        ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ material- ┆ 551064006 ┆ 9444      ┆ {9444,"ma ┆ … ┆ ["jverkoe ┆ Title:    ┆ material- ┆ material │
│ component ┆           ┆           ┆ terial-co ┆   ┆ y", "brya ┆ [AppBar]  ┆ component ┆ -compone │
│ s/materia ┆           ┆           ┆ mponents- ┆   ┆ noltman"] ┆ Fix swipe ┆ s         ┆ nts-ios  │
│ l-c…      ┆           ┆           ┆ ios…      ┆   ┆           ┆ to g…     ┆           ┆          │
│ dlang/pho ┆ 551980198 ┆ 7355      ┆ {7355,"ph ┆ … ┆ ["wilzbac ┆ Title:    ┆ dlang     ┆ phobos   │
│ bos       ┆           ┆           ┆ obos","dl ┆   ┆ h", "etci ┆ Add       ┆           ┆          │
│           ┆           ┆           ┆ ang"}     ┆   ┆ mon",     ┆ initial   ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ "Geod2…   ┆ support   ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆ for…      ┆           ┆          │
│ patternfl ┆ 552466427 ┆ 686       ┆ {686,"pat ┆ … ┆ ["starrye ┆ Title:    ┆ patternfl ┆ patternf │
│ y/pattern ┆           ┆           ┆ ternfly-e ┆   ┆ yez024",  ┆ added fix ┆ y         ┆ ly-eleme │
│ fly-eleme ┆           ┆           ┆ lements", ┆   ┆ "LyndseyR ┆ for nav   ┆           ┆ nts      │
│ nts       ┆           ┆           ┆ "pa…      ┆   ┆ "]        ┆ prior…    ┆           ┆          │
│ NixOS/nix ┆ 553379763 ┆ 78253     ┆ {78253,"n ┆ … ┆ ["vbgl",  ┆ Title:    ┆ NixOS     ┆ nixpkgs  │
│ pkgs      ┆           ┆           ┆ ixpkgs"," ┆   ┆ "jpas"]   ┆ coqPackag ┆           ┆          │
│           ┆           ┆           ┆ NixOS"}   ┆   ┆           ┆ es.CoLoR: ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆ 1.4.…     ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ okta/odys ┆ 118371419 ┆ 1399      ┆ {1399,"od ┆ … ┆ ["edburye ┆ Title:    ┆ okta      ┆ odyssey  │
│ sey       ┆ 5         ┆           ┆ yssey","o ┆   ┆ negren-ok ┆ Fix:      ┆           ┆          │
│           ┆           ┆           ┆ kta"}     ┆   ┆ ta",      ┆ Table     ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ "jeffbe…  ┆ sort      ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆ indicat…  ┆           ┆          │
│ rust-bitc ┆ 118805139 ┆ 924       ┆ {924,"rus ┆ … ┆ ["sanket1 ┆ Title:    ┆ rust-bitc ┆ rust-bit │
│ oin/rust- ┆ 6         ┆           ┆ t-bitcoin ┆   ┆ 729", "dr ┆ Improveme ┆ oin       ┆ coin     │
│ bitcoin   ┆           ┆           ┆ ","rust-b ┆   ┆ -orlovsky ┆ nts to    ┆           ┆          │
│           ┆           ┆           ┆ itc…      ┆   ┆ "]        ┆ taproot…  ┆           ┆          │
│ funcx-faa ┆ 118811103 ┆ 747       ┆ {747,"fun ┆ … ┆ ["sirosen ┆ Title:    ┆ funcx-faa ┆ funcX    │
│ s/funcX   ┆ 4         ┆           ┆ cX

In [15]:
pr_df = pr_df.rename({"field_0": "parent_repo", "field_1": "child_repo"})

In [16]:
pr_df["parent_repo"].value_counts(sort=True)

parent_repo,count
str,u32
"""apache""",8924
"""kubernetes""",4527
"""MicrosoftDocs""",4115
"""NixOS""",4103
"""Azure""",3799
…,…
"""webview""",1
"""DevShiftTeam""",1
"""geoffrich""",1


In [17]:
#let's filter for columns that we will use in the model
pr_df = pr_df.select(
    ['repo',
     'parent_repo',
     'child_repo',
     'issue_id',
     'issue_number',
     'issue',
     'text_size',
     'usernames',
    ])
pr_df

repo,parent_repo,child_repo,issue_id,issue_number,issue,text_size,usernames
str,str,str,i64,i64,str,i64,str
"""kaisermann/svelte-i18n""","""kaisermann""","""svelte-i18n""",550510104,40,"""Title: WIP - v3 username_0: """,2398,"""[""kaisermann"", ""elbourki""]"""
"""material-components/material-c…","""material-components""","""material-components-ios""",551064006,9444,"""Title: [AppBar] Fix swipe to g…",355,"""[""jverkoey"", ""bryanoltman""]"""
"""dlang/phobos""","""dlang""","""phobos""",551980198,7355,"""Title: Add initial support for…",4306,"""[""wilzbach"", ""etcimon"", ""Geod2…"
"""patternfly/patternfly-elements""","""patternfly""","""patternfly-elements""",552466427,686,"""Title: added fix for nav prior…",1995,"""[""starryeyez024"", ""LyndseyR""]"""
"""NixOS/nixpkgs""","""NixOS""","""nixpkgs""",553379763,78253,"""Title: coqPackages.CoLoR: 1.4.…",2544,"""[""vbgl"", ""jpas""]"""
…,…,…,…,…,…,…,…
"""okta/odyssey""","""okta""","""odyssey""",1183714195,1399,"""Title: Fix: Table sort indicat…",431,"""[""edburyenegren-okta"", ""jeffbe…"
"""rust-bitcoin/rust-bitcoin""","""rust-bitcoin""","""rust-bitcoin""",1188051396,924,"""Title: Improvements to taproot…",2842,"""[""sanket1729"", ""dr-orlovsky""]"""
"""funcx-faas/funcX""","""funcx-faas""","""funcX""",1188111034,747,"""Title: Mild clarification of d…",427,"""[""sirosen"", ""khk-globus""]"""


In [18]:
#SO we have just "usernames" that take part in "events" and "content", so I assume that the first name in "usernames" is the person
#who created the issue, and the last one who made the PR (although in practice thats the person who merged the PR)
#anyway in real case scenario we would have to find out which name comes from the author of Pull Request
#and would have to call the github API for data from that user
#to simplify the things let's just turn the user names into labels
#and join the labels with index of the kaggle dataset with users info, this way we can quickly mock the process of calling the Github repo for real user data

pr_df = pr_df.with_columns(pl.col("usernames").str.replace_all("\[","").str.replace_all("\]","").str.replace_all("\"","").str.split(by=","))

<>:8: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\]'
<>:8: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\]'
/var/folders/rx/9sb4r21x61798rxmm_5ds91c0000gn/T/ipykernel_38669/3368682134.py:8: SyntaxWarning: invalid escape sequence '\['
  pr_df = pr_df.with_columns(pl.col("usernames").str.replace_all("\[","").str.replace_all("\]","").str.replace_all("\"","").str.split(by=","))
/var/folders/rx/9sb4r21x61798rxmm_5ds91c0000gn/T/ipykernel_38669/3368682134.py:8: SyntaxWarning: invalid escape sequence '\]'
  pr_df = pr_df.with_columns(pl.col("usernames").str.replace_all("\[","").str.replace_all("\]","").str.replace_all("\"","").str.split(by=","))


In [19]:
pr_df = pr_df.with_columns(users = pl.col("usernames").list.last())

In [20]:
pr_df["users"].value_counts(sort=True).shape

(176861, 2)

In [21]:
pr_df

repo,parent_repo,child_repo,issue_id,issue_number,issue,text_size,usernames,users
str,str,str,i64,i64,str,i64,list[str],str
"""kaisermann/svelte-i18n""","""kaisermann""","""svelte-i18n""",550510104,40,"""Title: WIP - v3 username_0: """,2398,"[""kaisermann"", "" elbourki""]",""" elbourki"""
"""material-components/material-c…","""material-components""","""material-components-ios""",551064006,9444,"""Title: [AppBar] Fix swipe to g…",355,"[""jverkoey"", "" bryanoltman""]",""" bryanoltman"""
"""dlang/phobos""","""dlang""","""phobos""",551980198,7355,"""Title: Add initial support for…",4306,"[""wilzbach"", "" etcimon"", … "" jacob-carlborg""]",""" jacob-carlborg"""
"""patternfly/patternfly-elements""","""patternfly""","""patternfly-elements""",552466427,686,"""Title: added fix for nav prior…",1995,"[""starryeyez024"", "" LyndseyR""]",""" LyndseyR"""
"""NixOS/nixpkgs""","""NixOS""","""nixpkgs""",553379763,78253,"""Title: coqPackages.CoLoR: 1.4.…",2544,"[""vbgl"", "" jpas""]",""" jpas"""
…,…,…,…,…,…,…,…,…
"""okta/odyssey""","""okta""","""odyssey""",1183714195,1399,"""Title: Fix: Table sort indicat…",431,"[""edburyenegren-okta"", "" jeffbelser-okta""]",""" jeffbelser-okta"""
"""rust-bitcoin/rust-bitcoin""","""rust-bitcoin""","""rust-bitcoin""",1188051396,924,"""Title: Improvements to taproot…",2842,"[""sanket1729"", "" dr-orlovsky""]",""" dr-orlovsky"""
"""funcx-faas/funcX""","""funcx-faas""","""funcX""",1188111034,747,"""Title: Mild clarification of d…",427,"[""sirosen"", "" khk-globus""]",""" khk-globus"""


In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
le = LabelEncoder()
pr_df = pr_df.with_columns(pl.col("users").map_batches(le.fit_transform).alias("mock_number"))
pr_df.head()

repo,parent_repo,child_repo,issue_id,issue_number,issue,text_size,usernames,users,mock_number
str,str,str,i64,i64,str,i64,list[str],str,i64
"""kaisermann/svelte-i18n""","""kaisermann""","""svelte-i18n""",550510104,40,"""Title: WIP - v3 username_0: """,2398,"[""kaisermann"", "" elbourki""]",""" elbourki""",52812
"""material-components/material-c…","""material-components""","""material-components-ios""",551064006,9444,"""Title: [AppBar] Fix swipe to g…",355,"[""jverkoey"", "" bryanoltman""]",""" bryanoltman""",38978
"""dlang/phobos""","""dlang""","""phobos""",551980198,7355,"""Title: Add initial support for…",4306,"[""wilzbach"", "" etcimon"", … "" jacob-carlborg""]",""" jacob-carlborg""",66742
"""patternfly/patternfly-elements""","""patternfly""","""patternfly-elements""",552466427,686,"""Title: added fix for nav prior…",1995,"[""starryeyez024"", "" LyndseyR""]",""" LyndseyR""",14002
"""NixOS/nixpkgs""","""NixOS""","""nixpkgs""",553379763,78253,"""Title: coqPackages.CoLoR: 1.4.…",2544,"[""vbgl"", "" jpas""]",""" jpas""",72876


In [ ]:
#TODO: join the pr_df with github users dataset from kaggle
#create a dataset of "available" Good First Issues - we can mock also this dataset
#instead of calling github API for current/open Good First Issues, we can just re-use the data from the datasets we used so far
#build model that will compare New Users to Old Users
#build model comparing Old Issues with Open Issues

In [25]:
pr_df.write_parquet("data/intermediate_data/pr_df.parquet")
pr_df.shape

(414185, 10)